# Implementação do deep Q-learning aplicado aos jogos CartPole-v1 e LunarLander-v2

## **CartPole-v1**

O objetivo do programa é treinar um agente para resolver o problema do **CartPole-v1**, onde o agente precisa aprender a equilibrar um bastão (pole) em cima de um carrinho (cart). O agente utiliza o algoritmo *Deep *Q-Learning** para maximizar a recompensa acumulada.

---



<div style="display: flex; justify-content: center; align-items: center;">
    <img src="CartPole1.png" alt="Decision Tree 1" style="width: 45%; margin-right: 10px;"/>
    <img src="CartPole2.png" alt="Decision Tree 2" style="width: 45%;"/>
</div>


# **Etapas do Programa**

# 1. **Instalação de Dependências**
# 2. **Importação de Dependências**
   - **Bibliotecas Utilizadas:**
     - `gym`: Para criar e interagir com o ambiente do CartPole-v1.
     - `numpy`: Para operações numéricas.

In [ ]:
!pip install gym numpy tensorflow matplotlib
!pip install Box2D
!pip install --upgrade pip setuptools wheel


In [ ]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from collections import deque
import random

# 2. **Configuração do Ambiente**
   - Cria o ambiente `CartPole-v1`:
     ```python
     env = gym.make("CartPole-v1", render_mode="human")
     ```
   - O ambiente possui:
     - Estados contínuos (posição, velocidade, ângulo do pole).
     - Ações discretas (mover o carrinho para a esquerda ou direita). Ou seja, 3 possibilidades (parado, esquerda e direita).

In [ ]:
# Criação do ambiente CartPole-v1 e habilitação da renderização para visualização
env = gym.make("CartPole-v1", render_mode="human")

# Obtém o número de variáveis que descrevem o estado do ambiente (posição, velocidade, etc.)
state_size = env.observation_space.shape[0]

# Obtém o número de ações possíveis (esquerda ou direita)
action_size = env.action_space.n

# 3. **Hiperparâmetros e criação da rede neural**

Este trecho de código configura os **hiperparâmetros** e define uma **rede neural** para estimar a função $ Q(s, a) $, essencial para o treinamento do agente no contexto do Deep Q-Learning.

---

## **Hiperparâmetros**

Os hiperparâmetros controlam o comportamento do algoritmo de aprendizado e são configurados no início do programa:

1. **`gamma = 0.99` (Fator de desconto):**
   - Determina o peso das recompensas futuras.
   - Valores próximos de 1 priorizam recompensas de longo prazo.

2. **`epsilon = 1.0` (Taxa de exploração inicial):**
   - Controla a probabilidade de o agente tomar ações aleatórias.
   - Alta no início para incentivar a exploração do ambiente.

3. **`epsilon_min = 0.01` (Taxa mínima de exploração):**
   - Define o limite mínimo de $ \epsilon $, evitando que o agente pare completamente de explorar.

4. **`epsilon_decay = 0.995` (Taxa de decaimento de epsilon):**
   - Reduz gradualmente $ \epsilon $ ao longo do treinamento.
   - Promove a transição de exploração para exploração.

5. **`learning_rate = 0.001` (Taxa de aprendizado):**
   - Controla o quão rápido a rede neural ajusta seus pesos durante o treinamento.

6. **`batch_size = 64` (Tamanho do mini-batch):**
   - Número de amostras usadas em cada passo de treinamento da rede neural.

7. **`replay_buffer_size = 2000` (Tamanho do Replay Buffer):**
   - Capacidade do buffer que armazena as experiências do agente (estado, ação, recompensa, próximo estado).

8. **`episodes = 50` (Número de episódios):**
   - Número total de rodadas de treinamento que o agente realizará.

---

## **Rede Neural para Estimar $ Q(s, a) $**

A função `build_model` define uma rede neural que aproxima a função valor-ação \( Q(s, a) \). Essa rede é usada para decidir as ações do agente com base no estado atual.

### **Arquitetura da Rede**
1. **Entrada:**
   - O tamanho da entrada é definido por `input_dim=state_size`, correspondente ao número de variáveis do estado.

2. **Camadas Ocultas:**
   - Duas camadas densas (fully connected) com 24 neurônios cada e função de ativação ReLU:
     - **ReLU (Rectified Linear Unit):** Permite que a rede modele relações não lineares entre estado e ações.

3. **Saída:**
   - Uma camada densa com `action_size` neurônios (um para cada ação possível).
   - Função de ativação **linear**: Retorna valores contínuos representando $ Q(s, a) $ para cada ação.

4. **Compilação:**
   - **Otimizador:** Adam, para ajustar os pesos da rede.
   - **Função de perda:** Mean Squared Error (MSE), que mede o erro entre $ Q(s, a) $ previsto e o alvo calculado.

### **Código:**
```python
def build_model():
    model = Sequential([
        Dense(24, activation='relu', input_dim=state_size),  # Primeira camada oculta
        Dense(24, activation='relu'),                       # Segunda camada oculta
        Dense(action_size, activation='linear')             # Camada de saída
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse')  # Compilação da rede
    return model

In [ ]:
# Hiperparâmetros
gamma = 0.99                # Fator de desconto
epsilon = 1.0               # Taxa de exploração inicial
epsilon_min = 0.01          # Taxa mínima de exploração
epsilon_decay = 0.995       # Taxa de decaimento de epsilon
learning_rate = 0.001       # Taxa de aprendizado
batch_size = 64             # Tamanho do mini-batch
replay_buffer_size = 2000   # Tamanho do Replay Buffer
episodes = 50              # Número de episódios

# Rede Neural para estimar Q(s, a)
def build_model():
    model = Sequential([
        Dense(24, activation='relu', input_dim=state_size),
        Dense(24, activation='relu'),
        Dense(action_size, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse')
    return model


# 4. **Inicialização do Modelo e Replay Buffer**

### **Modelos:**
- **Rede Principal (`model`):**
  - Estima $Q(s, a)$ com base nos estados do ambiente.
- **Rede Alvo (`target_model`):**
  - É usada para calcular $\max Q(s', a')$, estabilizando o treinamento.
  - Inicializada com os mesmos pesos da rede principal.
  ```python
  model = build_model()
  target_model = build_model()
  target_model.set_weights(model.get_weights())
  ```

### **Replay Buffer:**
- Um buffer circular usado para armazenar experiências do agente ($s, a, r, s', done$).
- Permite que a rede aprenda de amostras aleatórias, reduzindo a correlação entre as atualizações.
  ```python
  replay_buffer = deque(maxlen=replay_buffer_size)
  ```

---

## **2. Escolha da Ação (Política $\epsilon$-greedy)**

A função `choose_action` implementa a política $\epsilon$-greedy para decidir a próxima ação do agente:
1. **Exploração (Ação Aleatória):**
   - Com probabilidade $\epsilon$, o agente escolhe uma ação aleatória para explorar o ambiente.
2. **Exploração (Ação Baseada em Q-Value):**
   - Com probabilidade $1 - \epsilon$, o agente escolhe a ação com o maior valor $Q(s, a)$, prevista pela rede principal.
   ```python
   def choose_action(state):
       if np.random.rand() <= epsilon:
           return random.choice(range(action_size))  # Ação aleatória
       q_values = model.predict(state[np.newaxis, :], verbose=0)
       return np.argmax(q_values[0])  # Ação com maior valor Q
   ```

---

## **3. Atualização da Rede Alvo**

A rede alvo é atualizada periodicamente para refletir os pesos da rede principal. Isso reduz oscilações durante o treinamento:
```python
def update_target_model():
    target_model.set_weights(model.get_weights())
```

---

## **4. Replay Buffer para Treinamento**

A função `replay` usa experiências armazenadas no Replay Buffer para treinar a rede principal.

### **Etapas:**
1. **Checar Disponibilidade de Dados:**
   - Aguarda o Replay Buffer ter experiências suficientes para realizar o treinamento.
   ```python
   if len(replay_buffer) < batch_size:
       return
   ```

2. **Amostragem Aleatória:**
   - Seleciona um mini-batch aleatório do buffer.
   ```python
   mini_batch = random.sample(replay_buffer, batch_size)
   states, actions, rewards, next_states, dones = zip(*mini_batch)
   ```

3. **Cálculo do Valor-Alvo:**
   - O valor-alvo é calculado com base na recompensa imediata ($R$) e no melhor $Q(s', a')$ estimado pela rede alvo.
   ```python
   target = rewards[i]
   if not dones[i]:
       target += gamma * np.max(q_values_next[i])
   q_values[i][actions[i]] = target
   ```

4. **Treinamento do Modelo:**
   - A rede principal é ajustada para minimizar a diferença entre o $Q(s, a)$ previsto e o valor-alvo.
   ```python
   model.fit(states, q_values, epochs=1, verbose=0, batch_size=batch_size)
   ```

5. **Decaimento de $\epsilon$:**
   - A taxa de exploração ($\epsilon$) é reduzida gradualmente para priorizar a exploração.
   ```python
   if epsilon > epsilon_min:
       epsilon *= epsilon_decay
   ```

---

## **Resumo**
1. **Modelos:** Inicializa a rede principal e a rede alvo para estimar $Q(s, a)$.
2. **Replay Buffer:** Armazena experiências do agente para aprendizado off-policy.
3. **Política $\epsilon$-greedy:** Alterna entre explorar o ambiente e explorar as ações com maior valor $Q$.
4. **Treinamento:** Usa experiências armazenadas para ajustar a rede principal e reduzir a perda.
5. **Estabilização:** A rede alvo é atualizada periodicamente com os pesos da rede principal.

In [ ]:
# Inicializar modelo e Replay Buffer
model = build_model()
target_model = build_model()  # Rede Alvo
target_model.set_weights(model.get_weights())  # Inicializar com os mesmos pesos
replay_buffer = deque(maxlen=replay_buffer_size)

# Função para selecionar a ação (epsilon-greedy)
def choose_action(state):
    if np.random.rand() <= epsilon:
        return random.choice(range(action_size))  # Ação aleatória
    q_values = model.predict(state[np.newaxis, :], verbose=0)
    return np.argmax(q_values[0])  # Ação com maior valor Q

# Atualizar os pesos da rede alvo
def update_target_model():
    target_model.set_weights(model.get_weights())

# Replay Buffer para treino
def replay():
    global epsilon
    if len(replay_buffer) < batch_size:
        return  # Esperar até que o Replay Buffer tenha tamanho suficiente

    mini_batch = random.sample(replay_buffer, batch_size)
    states, actions, rewards, next_states, dones = zip(*mini_batch)

    states = np.array(states)
    next_states = np.array(next_states)
    rewards = np.array(rewards)
    dones = np.array(dones)

    q_values = model.predict(states, verbose=0)
    q_values_next = target_model.predict(next_states, verbose=0)

    for i in range(batch_size):
        target = rewards[i]
        if not dones[i]:
            target += gamma * np.max(q_values_next[i])
        q_values[i][actions[i]] = target

    model.fit(states, q_values, epochs=1, verbose=0, batch_size=batch_size)

    if epsilon > epsilon_min:
        epsilon *= epsilon_decay


# 4. **Treinamento do Agente**


In [ ]:
for e in range(episodes):  # Loop principal que percorre todos os episódios configurados
    state, _ = env.reset()  # Reinicia o ambiente e obtém o estado inicial
    total_reward = 0  # Inicializa a recompensa total para o episódio

    for time in range(500):  # Limita o número de interações dentro de um episódio
        action = choose_action(state)  # Escolhe uma ação com a política epsilon-greedy
        next_state, reward, done, _, _ = env.step(action)  # Executa a ação no ambiente
        next_state = np.array(next_state)  # Converte o próximo estado para um array NumPy

        # Armazena a experiência atual no Replay Buffer
        replay_buffer.append((state, action, reward, next_state, done))
        state = next_state  # Atualiza o estado atual para o próximo estado
        total_reward += reward  # Soma a recompensa ao total do episódio

        if done:  # Verifica se o episódio terminou
            print(f"Episódio {e+1}/{episodes}, Pontuação: {total_reward}")  # Exibe a pontuação final
            break  # Interrompe o loop interno se o episódio terminou

        replay()  # Treina a rede principal com experiências do Replay Buffer

    # Atualiza a rede alvo a cada 10 episódios para estabilizar o treinamento
    if e % 10 == 0:
        update_target_model()


# 5. **Testar o agente treinado**


In [ ]:

# Testar o agente treinado
for _ in range(5):
    state, _ = env.reset()  # Apenas o estado inicial
    state = np.array(state)
    done = False
    while not done:
        action = np.argmax(model.predict(state[np.newaxis, :], verbose=0))
        state, _, done, _, _ = env.step(action)
        env.render()

env.close()


# **LunarLander-v2**

O programa treina um agente para resolver o problema do **LunarLander-v2**, onde o objetivo é pousar suavemente um módulo lunar entre duas bandeiras. O agente aprende utilizando **Deep Q-Learning**.

---

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="LunarLander1.png" alt="Decision Tree 1" style="width: 45%; margin-right: 10px;"/>
    <img src="LunarLander2.png" alt="Decision Tree 2" style="width: 45%;"/>
</div>

In [ ]:
import gym
import numpy as np

# Configuração do ambiente
env = gym.make("LunarLander-v2", render_mode="human")

# Hiperparâmetros
learning_rate = 0.1
discount_factor = 0.99
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01
episodes = 1000

# Discretização do espaço de estados
state_bins = [10] * 8  # Divisões para cada uma das 8 dimensões do estado
state_bounds = list(zip(env.observation_space.low, env.observation_space.high))
state_bins = [
    np.linspace(bounds[0], bounds[1], num - 1)
    for bounds, num in zip(state_bounds, state_bins)
]

# Tabela Q
action_size = env.action_space.n
state_size = tuple(len(bins) + 1 for bins in state_bins)
q_table = np.zeros(state_size + (action_size,))

# Função para discretizar estados
def discretize_state(state):
    discretized = [
        np.digitize(state[i], state_bins[i]) for i in range(len(state))
    ]
    return tuple(discretized)

# Treinamento do agente
for episode in range(episodes):
    state, _ = env.reset()
    state = discretize_state(state)
    total_reward = 0

    done = False
    while not done:
        # Escolha da ação (epsilon-greedy)
        if np.random.rand() < epsilon:
            action = np.random.choice(action_size)
        else:
            action = np.argmax(q_table[state])

        # Executa a ação
        next_state, reward, done, _, _ = env.step(action)
        next_state = discretize_state(next_state)

        # Atualização da Tabela Q
        best_next_action = np.argmax(q_table[next_state])
        q_table[state][action] += learning_rate * (
            reward + discount_factor * q_table[next_state][best_next_action]
            - q_table[state][action]
        )

        state = next_state
        total_reward += reward

    # Atualizar epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f"Episódio {episode + 1}/{episodes}, Pontuação: {total_reward}")


### Bonus: Stacking: Empilhar múltiplos frames (ex.: 4 últimos frames) para capturar o movimento. Essa estratégia é importante para jogos dinâmicos.


<img src="StreetFighter_1.png" alt="Iris dataset" width="300"/>

(https://www.youtube.com/watch?v=3SLNbON-upI)

<img src="StreetFighter_2.png" alt="Iris dataset" width="500"/>

(https://www.youtube.com/watch?v=3SLNbON-upI)
